In [1]:
import sys
import os

sys.path.append('..')

import utilities.functions as functions

from utilities.functions import (
    load_data,
    check_key_uniqueness,
    save_parquet,
    merge_df,
    load_orders,
    process_orders_pandas,
)

Load the data --- se necessario salvar em stage - neste momento o estara comentado

In [2]:
URL_CONSUMER = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/consumer.csv.gz"
df_consumer = load_data(URL_CONSUMER)[["customer_id", "active","created_at"]]

In [3]:
URL_RESTAURANT ="https://data-architect-test-source.s3-sa-east-1.amazonaws.com/restaurant.csv.gz"
df_restaurant= load_data(URL_RESTAURANT)

In [4]:
ab_test_url = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/ab_test_ref.tar.gz"
df_ab = load_data(ab_test_url)

In [5]:
#save_parquet(df_consumer, "stage", "df_consumer.parquet")
#save_parquet(df_restaurant, "stage", "df_restaurant.parquet")
#save_parquet(df_consumer, "stage", "df_consumer.parquet")

Bronze layer - verify duplicates e nulo. e se necessario remover

In [6]:
check_key_uniqueness(df_consumer, ["customer_id","active","created_at"])

✅ Colunas ['customer_id', 'active', 'created_at'] são NOT NULL e UNIQUE.


(True, None, None, None)

In [7]:
check_key_uniqueness(df_restaurant, ["id"])

✅ Colunas ['id'] são NOT NULL e UNIQUE.


(True, None, None, None)

In [8]:
check_key_uniqueness(df_ab, ["customer_id","is_target"])
print(df_ab[df_ab["customer_id"].isna()])


❌ Colunas ['customer_id', 'is_target'] contêm valores nulos.

Soma de nulos por coluna:
customer_id    1
is_target      0
dtype: int64

Índices com nulos:
[81149]
      customer_id is_target
81149         NaN    target


In [9]:
df_ab_np = df_ab[~df_ab["customer_id"].isna()]

Antes de carregar a base ordens sera definido o publico todal, e da base de ordem serao filtrados somentes os clientes elegiceis

In [10]:
df_publico=merge_df(df_ab_np,df_consumer,['customer_id'],'outer')

In [11]:
clientes_mes = (
    df_publico.groupby(['active', 'is_target'], dropna=False)
              ['customer_id']
              .nunique()
              .reset_index(name='numero_clientes_distintos')
)
clientes_mes

,active,is_target,numero_clientes_distintos
0,False,control,713
1,False,target,882
2,True,control,359700
3,True,target,444861
4,NaN,control,129
5,NaN,target,181


In [12]:
df_publico = df_publico.dropna(subset=['active'])

In [13]:
clientes_mes = (
    df_publico.groupby(['active', 'is_target'], dropna=False)
              ['customer_id']
              .nunique()
              .reset_index(name='numero_clientes_distintos')
)
clientes_mes

,active,is_target,numero_clientes_distintos
0,False,control,713
1,False,target,882
2,True,control,359700
3,True,target,444861


Publico definido e todos os clientes marcados no teste a/b e existentes na base de clientes

In [14]:
save_parquet(df_publico, "silver", "df_publico.parquet")

💾 SALVO: dados/silver/df_publico.parquet
📁 Local: /Users/maceli/ifood_case/code/notebook/dados/silver/df_publico.parquet


PosixPath('dados/silver/df_publico.parquet')

Da base de ordens serao filtrados todos os clientes com orden nos meses de de dezembro e janeiro

In [ ]:
df_ab=df_ab.head(100)

In [15]:
URL_ORDERS = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/order.json.gz"

COLUMNS_TO_DROP = [
    'cpf','customer_name','delivery_address_city','delivery_address_country',
    'delivery_address_district','delivery_address_external_id',
    'delivery_address_latitude','delivery_address_longitude',
    'delivery_address_state','delivery_address_zip_code','items',
    'merchant_latitude','merchant_longitude','merchant_timezone',
    'order_scheduled','order_scheduled_date'
]

customer_ids = df_ab["customer_id"].astype(str).unique()

df_orders = load_orders(
    url=URL_ORDERS,
    customer_ids=customer_ids,
    columns_to_drop=COLUMNS_TO_DROP
)

df_orders.head()


,customer_id,merchant_id,order_created_at,order_id,order_total_amount,origin_platform
0,7ba88a68bb2a3504c6bd37a707af57a0b8d6e110a551c7...,a992a079a651e699d9149423761df2427c0e3af0a2a1b5...,2019-01-17T22:50:06.000Z,33e0612d62e5eb42aba15b58413137e441fbe906de2feb...,46.0,ANDROID
1,078acecdcf7fa89d356bfa349f14a8219db1ee161ce28a...,5152f28ee0518b8803ccf0a4096eb2ff8b81e9491861c9...,2019-01-17T17:51:26.000Z,148c4353a2952f3fe7973547283265eb22b575fb712ed2...,104.5,ANDROID
2,0e38a3237b5946e8ab2367b4f1a3ae6e77f1e215bc760c...,b6096419455c35d06105a5ef0d25c51f9dd40e1e99ac33...,2019-01-17T22:53:47.000Z,c37e495a91b498bb7b70a9e09ac115d0cdd443f152dc11...,35.0,IOS
3,cab1a004b7206d07910092c515a79834fea0a03d7d9054...,082bfdcdf6ccdc343e3c4d25ee376b5b6ca7e96ad8b04e...,2019-01-17T23:56:53.000Z,b4df94142d21354611247da9ca94f870c09b93989b531a...,40.8,IOS
4,aa7edf5b166b8c843aec3b96dc561222888734f3879123...,d7adb764bac29ccb77fb8f746ffbd531bf05ec30a7e130...,2019-01-17T23:40:53.000Z,4ff64b33b272c1886df21b63272220af6a82d1667dba70...,48.5,ANDROID


In [16]:
print(check_key_uniqueness(df_orders, ["customer_id","merchant_id","order_id"]))
print(check_key_uniqueness(df_orders, ["customer_id","merchant_id","order_id","order_created_at"]))

❌ Colunas ['customer_id', 'merchant_id', 'order_id'] possuem duplicações.
(False,                                                customer_id  \
1192509  c94d0872922e87c7f23669afcce3cc700d82ed75080a59...   
1192510  1e2af3429ee49b319a095258707980a7f1da2c2f1b6ca9...   
1192511  9a68122c178c32840eef9421530a375629d586ef9a7643...   
1192512  afdf6cab94f0e58e8a03940bb23243dc73263ad217205c...   
1192513  e9f79cc65b905e1e8cae0687605be386bc0275f6ec46ad...   
...                                                    ...   
3662316  648ae0e610811af0fccbe557b9a63a55c6e46adeeceb0b...   
3662317  5cab7f42316c5815d151d1fd0eebaecf9e6e53681257f0...   
3662318  1e91e110ba83f466ddbdb8ea448940e39e4e5ce16925e5...   
3662319  588becd71bc59b9a17ffcafe5823ce77777f308296f6f0...   
3662320  50862fb1670635160c98cd292768893dd65df05e5ae38e...   

                                               merchant_id  \
1192509  7e1b40dd1d08526b316dbbc60be12806ef7a5b7dd8278e...   
1192510  d1b5d9f45379a82adab72900747e635d4d9ee40d

In [17]:
#save_parquet(df_orders, "stage", "df_orders.parquet")

Add orders para a base de publico

In [18]:
df_publico_orders=merge_df(df_publico,df_orders,['customer_id'],'inner')

In [19]:
df_publico_orders.head()

,customer_id,is_target,active,created_at,merchant_id,order_created_at,order_id,order_total_amount,origin_platform
0,000009e29b12d8c5b9c9579e21384320c2df3bd935f19e...,control,True,2018-04-06T00:05:23.543Z,75168a16437068a463d8c825e7c4638128a028f1745245...,2019-01-01T20:31:38.000Z,c623211eb5dda070416069c51f36e4370260e94ee6a45e...,28.5,ANDROID
1,000009e29b12d8c5b9c9579e21384320c2df3bd935f19e...,control,True,2018-04-06T00:05:23.543Z,f6b76403a124780d6fc242de3d9c398eec9b436d769b3f...,2019-01-25T23:26:38.000Z,0b6b7456c4bf57f56f97c752b1d500e8ad5b3f535494b8...,30.4,ANDROID
2,000009e29b12d8c5b9c9579e21384320c2df3bd935f19e...,control,True,2018-04-06T00:05:23.543Z,f6b76403a124780d6fc242de3d9c398eec9b436d769b3f...,2019-01-16T01:31:20.000Z,a31b414ef2936435ed56b98107d714c8f2819a82fcb782...,31.4,ANDROID
3,000009e29b12d8c5b9c9579e21384320c2df3bd935f19e...,control,True,2018-04-06T00:05:23.543Z,ab0ecc0f539748be384ff8456b251224ede4f833c8058f...,2019-01-10T15:37:46.000Z,b83be3a923ab36e459500fdb5f2afb3ffa887429bb1cbf...,24.9,ANDROID
4,000009e29b12d8c5b9c9579e21384320c2df3bd935f19e...,control,True,2018-04-06T00:05:23.543Z,659a196ad800c788115707631267ee6a232915cb855180...,2019-01-25T14:57:46.000Z,a6469ab902d0da5d6a258cc6ca1c805dd6b166148ae402...,24.0,ANDROID


Construcao de chave unica, e sumarizacoes visao cliente

In [20]:
df_publico_orders=df_publico_orders.head(100)

In [21]:
df_publico=process_orders_pandas(df_publico_orders)

In [22]:
save_parquet(df_publico, "silver", "df_publico.parquet")

💾 SALVO: dados/silver/df_publico.parquet
📁 Local: /Users/maceli/ifood_case/code/notebook/dados/silver/df_publico.parquet


PosixPath('dados/silver/df_publico.parquet')

Uma linha por cliente, com as variaveis necessarias

In [23]:
df_pub_un = df_publico[["customer_id",'origin_platform',"is_target", "order_created_month", "num_pedidos_mes", "num_pedidos_hist",'total_amount_mes','ticket_medio']].drop_duplicates().reset_index(drop=True)
df_pub_un.head()

,customer_id,origin_platform,is_target,order_created_month,num_pedidos_mes,num_pedidos_hist,total_amount_mes,ticket_medio
0,000182788d43dc500c94ce7dfb6809b84f8561860efc53...,IOS,target,1,1,1,24.80,24.80
1,00016cfde8c0af0a4a3c6c31956cd0c5e3ef230054183e...,DESKTOP,control,1,1,1,51.98,51.98
2,0001484091f67071593b292cc04917bd6f4fc61c8de826...,IOS,control,1,1,1,43.70,43.70
3,0001274ea3bc24cee339c5bfe5c579c9175cb9a3a50503...,ANDROID,target,1,1,1,62.90,62.90
4,000126c9dafa16428eecae458c64435c8e894d24c48dec...,ANDROID,target,12,1,2,17.90,17.90


Salvar base visao cliente em gold layer

In [24]:
save_parquet(df_pub_un, "gold", "df_pub_un.parquet")

💾 SALVO: dados/gold/df_pub_un.parquet
📁 Local: /Users/maceli/ifood_case/code/notebook/dados/gold/df_pub_un.parquet


PosixPath('dados/gold/df_pub_un.parquet')